In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

import gpxpy
import gpxpy.gpx
import folium
import random

In [2]:
ROUTES_PATH = 'http://tosamara.ru/api/classifiers/routesAndStopsCorrespondence.xml'

In [3]:
req = requests.get(ROUTES_PATH)
tree = ET.fromstring(req.text)
tree

<Element 'routes' at 0x000001A45EE485E8>

In [4]:
routes = []
for child in tree:
    number = child[1].text
    direction = child[2].text
    type_id = int(child[3][0].text)
    performing = bool(child[4].text)
    geometry = child[-1].text.split(' ')
    routes.append({'number': number,
                  'direction': direction,
                  'type_id': type_id,
                  'performing' : performing,
                  'geometry' : geometry})

In [5]:
df_routes = pd.DataFrame.from_records(routes)
df_routes.head()

,number,direction,type_id,performing,geometry
0,1,Железнодорожный вокзал,1,True,"[53.384692,50.169369, 53.384749,50.169603, 53...."
1,1,Красная Глинка,1,True,"[53.187111,50.121445, 53.18814,50.122168, 53.1..."
2,21,Юнгородок,1,True,"[53.259224,50.216663, 53.259209,50.216672, 53...."
3,21,Барбошина Поляна,1,True,"[53.213223,50.291229, 53.213392,50.291594, 53...."
4,27,"кинотеатр ""Луч""",1,True,"[53.213391,50.247921, 53.217799,50.24232, 53.2..."


In [6]:
len(df_routes.number.unique())

169

In [7]:
def parse_geom(x):
    points = [(float(i.split(',')[0]),float(i.split(',')[1])) for i in x]
    edges = [(points[i-1],points[i]) for i in range(1, len(points))]
    return edges
df_routes['geo'] = df_routes['geometry'].apply(parse_geom)

In [9]:
global_edges = set()
for i in df_routes['geo'].values:
    global_edges = global_edges | set(i)

In [10]:
len(global_edges)

13310

In [11]:
points = {j for i in list(global_edges) for j in i}

ave_lat = sum(p[0] for p in points)/len(points)
ave_lon = sum(p[1] for p in points)/len(points)

# Load map centred on average coordinates
my_map = folium.Map(location=[ave_lat, ave_lon], zoom_start=10)

#fadd lines
# for k,v in work_day_10_dict.items():
for k in global_edges:
    folium.PolyLine(k, weight=3, opacity=1).add_to(my_map)

# Save map
my_map.save("./routes.html")
# my_map

In [12]:
df_routes.head()

,number,direction,type_id,performing,geometry,geo
0,1,Железнодорожный вокзал,1,True,"[53.384692,50.169369, 53.384749,50.169603, 53....","[((53.384692, 50.169369), (53.384749, 50.16960..."
1,1,Красная Глинка,1,True,"[53.187111,50.121445, 53.18814,50.122168, 53.1...","[((53.187111, 50.121445), (53.18814, 50.122168..."
2,21,Юнгородок,1,True,"[53.259224,50.216663, 53.259209,50.216672, 53....","[((53.259224, 50.216663), (53.259209, 50.21667..."
3,21,Барбошина Поляна,1,True,"[53.213223,50.291229, 53.213392,50.291594, 53....","[((53.213223, 50.291229), (53.213392, 50.29159..."
4,27,"кинотеатр ""Луч""",1,True,"[53.213391,50.247921, 53.217799,50.24232, 53.2...","[((53.213391, 50.247921), (53.217799, 50.24232..."


In [13]:
df_routes_perf = df_routes[df_routes.performing].drop(columns=['direction', 'performing', 'geometry'])
df_routes_perf.head()

,number,type_id,geo
0,1,1,"[((53.384692, 50.169369), (53.384749, 50.16960..."
1,1,1,"[((53.187111, 50.121445), (53.18814, 50.122168..."
2,21,1,"[((53.259224, 50.216663), (53.259209, 50.21667..."
3,21,1,"[((53.213223, 50.291229), (53.213392, 50.29159..."
4,27,1,"[((53.213391, 50.247921), (53.217799, 50.24232..."


In [14]:
req = requests.get('http://www.samaratrans.info/wiki/index.php/%D0%A1%D0%B0%D0%BC%D0%B0%D1%80%D0%B0_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_2')
html_text = req.text

In [15]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_text, 'html.parser')

In [16]:
collect_links = []
for j in soup.find_all('ul')[-6:]:
    for i in j.find_all('a'):
        try:
            if "маршруты" not in i['title'] and "расписания" not in i['title']:
                collect_links.append(i['href'])
        except KeyError:
            pass
len(collect_links)

199

In [17]:
def parse_add_inf(t):
    additional_information = t.find_all('td')
    add_inf = {}
    for i in range(0,14,2):
        add_inf[additional_information[i].text.strip()] = additional_information[i+1].text.strip()
    return add_inf


def parse_route_page(soup):
    try:
        tables = soup.find_all('table')
        route_name = tables[2].find_all('li')[0].text.split('\n\n')[0]
        return route_name, parse_add_inf(tables[7])
    except:
        return None, None

In [18]:
DOMAIN_LINK = 'http://www.samaratrans.info'

parsed_routes = {}

# for link in collect_links:
#     r = requests.get(DOMAIN_LINK+link)
#     s = BeautifulSoup(r.text, 'html.parser')
#     name, inf = parse_route_page(s)
#     parsed_routes[name] = inf
  

In [19]:
# with open('file.txt', 'w') as f:
#     f.write(str(parsed_routes))
with open('file.txt', 'r') as f:
    parsed_routes = eval(f.read())
# parsed_routes

In [20]:
type_dict = {int(i.split(' — ')[0]): i.split(' — ')[1] for i in '''1 — автобус, 2 — метрополитен, 3 — трамвай, 4 — троллейбус, 5 — электропоезд, 6 — речной транспорт'''.split(', ')}
type_dict

{1: 'автобус',
 2: 'метрополитен',
 3: 'трамвай',
 4: 'троллейбус',
 5: 'электропоезд',
 6: 'речной транспорт'}

In [21]:
routes_inf = []
exceptions = {}
for key, value in parsed_routes.items():
    try:
        key_splited = key.split(' ')
        number = key_splited[4]
        type_title = key_splited[1]
        schedule_days = { i.split(':')[0]: i.split(':')[1] 
                 for i in value['Режим работы'].split('По') if len(i.split(':')) > 1}
        intervals = value['Интервалы движения']
        vehicles = value['Подвижной состав']
        routes_inf.append([number, type_title, schedule_days, intervals, vehicles])
    except:
#         print(key, value)
        exceptions[key] = value

In [22]:
for key, value in exceptions.items():
    try:
        key_splited = key.split(' ')
        number = key_splited[4]
        type_title = key_splited[1]
        schedule_days = {value['Дни работы'] : value['Время работы'] }
        intervals = value['Интервалы движения']
        vehicles = value['Выпуск на линию']
        routes_inf.append([number, type_title, schedule_days, intervals, vehicles])
    except:
#         print(key, value)
        exceptions[key] = value

In [23]:
routes_inf.append(['7', 'Автобусный', {'Ежедневно': ' с 6 до 20 часов'}, 'По рабочим дням: 12-84 мин.По выходным дням: 68-164 мин.', 'ПАЗ-3203ПАЗ-3204ПАЗ-32054'])
routes_inf.append(['27', 'Автобусный', {'Ежедневно': ' с 6 до 20 часов'}, 'По рабочим дням: 12-44 мин.По субботам: 14-46 мин.По воскресеньям: 18-58 мин.', 'Hyundai County Kuzbas HDU2ПАЗ-3203ПАЗ-3204ПАЗ-32054'])
routes_inf.append(['59', 'Автобусный', {'Ежедневно': ' с 6 до 19 часов'}, '77 мин.', 'ПАЗ-3203'])
routes_inf.append(['59а', 'Автобусный', {'Ежедневно': ' с 6 до 19 часов'}, '77 мин.', 'ПАЗ-3203'])
routes_inf.append(['157', 'Автобусный', {'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по средам и пятницам(кроме дней массовых посещений кладбищ)': 'С 7 до 12 часов и с 15 до 20 часов'}, 'По рабочим дням: 30-35 мин.По выходным дням: 25-50 мин.', 'ЛиАЗ-5293.70МАЗ-206'])
routes_inf.append(['174', 'Автобусный', {'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по средам и пятницам(кроме дней массовых посещений кладбищ)': 'С 7 до 12 часов и с 16 до 19 часов'}, '45-50 мин.', 'МАЗ-206'])
routes_inf.append(['4', 'Трамвайный', {'Ежедневно': 'с 5 до 21 часа'}, '14 мин.', 'Tatra T3SU (Т-3) (одиночные вагоны)АКСМ-62103 (одиночные вагоны)'])
routes_inf.append(['4', 'Трамвайный', {' рабочим дням': ' с 6 до 23 часов', ' выходным дням': ' с 6 до 22 часов'}, 'По рабочим дням: 16 мин.По выходным дням: 14 мин.', 'Tatra T3SU (Т-3) (одиночные вагоны)АКСМ-62103 (одиночные вагоны)71-402 "СПЕКТР" (одиночные вагоны)'])

In [24]:
df_routes_inf = pd.DataFrame(routes_inf, columns=['number', 'type_title', 'schedule_days', 'intervals', 'vehicles'])
df_routes_inf.head()

,number,type_title,schedule_days,intervals,vehicles
0,1,Автобусный,{'Ежедневно': ' с 6 до 22 часов'},По рабочим дням: 8-20 мин.По выходным дням: 16...,ЛиАЗ-5293.70
1,3,Автобусный,{'Ежедневно': ' с 7 до 20 часов'},48-60 мин.,МАЗ-206
2,4,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},7-10 мин.,Автобусы малого класса
3,5д,Автобусный,{'Ежедневно': ' с 5 до 22 часов'},По рабочим дням: 10-20 мин.По выходным дням: 1...,ЛиАЗ-5293.70
4,6,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},По рабочим дням: 15-34 мин.По выходным дням: 1...,МАЗ-206


In [25]:
df_routes_inf.iloc[10].schedule_days = {'Ежедневно': 'С 7 до 20 часов'}
df_routes_inf.iloc[26].schedule_days = {'Ежедневно': 'С 6 до 20 часов'}

In [26]:
set([j for i in df_routes_inf.schedule_days.values for j in i.keys()])

{' выходным дням',
 ' рабочим дням',
 'В дни проведения мероприятий на стадионе "Самара Арена"Начало движения',
 'Выходные дни',
 'Ежедневно',
 'Пасха, Красная горка, Радоница, Троицкая родительская суббота, День Победы',
 'С 1 мая по 31 октября: по выходным и праздничным дням(кроме дней массовых посещений кладбищ)',
 'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по вторникам и четвергам(кроме дней массовых посещений кладбищ)',
 'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по средам и пятницам(кроме дней массовых посещений кладбищ)'}

Пасха, Красная горка, Радоница, Троицкая родительская суббота, День Победы 

Пасха - всегда воскресение
Красная горка- через неделю после Пасхи, тоже воскресение
Радоница - 9 дней после пасхи - вторник
Троицкая родительская суббота - суббота
День Победы - всегда выходной

In [27]:
def parse_time(text_time):
    words = text_time.strip().split(' ')
    if 'и ' in text_time:
        return list(range(int(words[1]), int(words[3]))) + list(range(int(words[7]), int(words[9])))
    return list(range(int(words[1]), int(words[3])))

def schedule_day(x):
    keys = x.schedule_days.keys()
    work_day, saturday, sunday, religion, arena, summer_weekend, summer_thu_tue, summer_wed_fri = [None for i in range(8)]
    if 'Ежедневно' in keys:
        work_day, saturday, sunday = [parse_time(x.schedule_days['Ежедневно']) for i in range(3)]
    if ' выходным дням' in keys:
        saturday, sunday = [parse_time(x.schedule_days[' выходным дням']) for i in range(2)]
    if 'Выходные дни' in keys:
        saturday, sunday = [parse_time(x.schedule_days['Выходные дни']) for i in range(2)]
    if ' рабочим дням' in keys:
        work_day = parse_time(x.schedule_days[' рабочим дням'])
    if 'В дни проведения мероприятий на стадионе "Самара Арена"Начало движения' in keys:
        arena = parse_time('с 15 до 23 часов')
    if 'Пасха, Красная горка, Радоница, Троицкая родительская суббота, День Победы' in keys:
        religion = parse_time(x.schedule_days['Пасха, Красная горка, Радоница, Троицкая родительская суббота, День Победы'])
    if 'С 1 мая по 31 октября: по выходным и праздничным дням(кроме дней массовых посещений кладбищ)' in keys:
        summer_weekend = parse_time(x.schedule_days['С 1 мая по 31 октября: по выходным и праздничным дням(кроме дней массовых посещений кладбищ)'])
    if 'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по вторникам и четвергам(кроме дней массовых посещений кладбищ)' in keys:
        summer_thu_tue = parse_time(x.schedule_days['С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по вторникам и четвергам(кроме дней массовых посещений кладбищ)'])
    if 'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по средам и пятницам(кроме дней массовых посещений кладбищ)' in keys:
        summer_wed_fri = parse_time(x.schedule_days['С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по средам и пятницам(кроме дней массовых посещений кладбищ)'])
    return pd.Series([work_day, saturday, sunday, religion, arena, summer_weekend, summer_thu_tue, summer_wed_fri])

df_routes_inf[['work_day', 'saturday', 'sunday', 'religion', 'arena', 'summer_weekend', 'summer_thu_tue', 'summer_wed_fri']] = df_routes_inf.apply(schedule_day, axis=1)

In [28]:
df_routes_inf.head()

,number,type_title,schedule_days,intervals,vehicles,work_day,saturday,sunday,religion,arena,summer_weekend,summer_thu_tue,summer_wed_fri
0,1,Автобусный,{'Ежедневно': ' с 6 до 22 часов'},По рабочим дням: 8-20 мин.По выходным дням: 16...,ЛиАЗ-5293.70,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None
1,3,Автобусный,{'Ежедневно': ' с 7 до 20 часов'},48-60 мин.,МАЗ-206,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]",None,None,None,None,None
2,4,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},7-10 мин.,Автобусы малого класса,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None
3,5д,Автобусный,{'Ежедневно': ' с 5 до 22 часов'},По рабочим дням: 10-20 мин.По выходным дням: 1...,ЛиАЗ-5293.70,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",None,None,None,None,None
4,6,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},По рабочим дням: 15-34 мин.По выходным дням: 1...,МАЗ-206,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None


In [29]:
df_routes_inf[df_routes_inf.religion.notna()]

,number,type_title,schedule_days,intervals,vehicles,work_day,saturday,sunday,religion,arena,summer_weekend,summer_thu_tue,summer_wed_fri
171,4к,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",20-40 мин.,1-2 машины,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
172,27к,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",10-20 мин.,4-6 машин,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
173,36д,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",20-30 мин.,3-4 машины,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
174,36ю,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",30 мин.,2 машины,None,None,None,"[8, 9, 10, 11, 12, 13]",None,None,None,None
175,63э,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",5-10 мин.,До 25 машин,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
176,67к,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",30-60 мин.,1-2 машины,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
177,77д,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",15-20 мин.,4-5 машин,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
178,129э,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",5-10 мин.,До 20 машин,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None


In [30]:
df_routes_inf.intervals = df_routes_inf.intervals.apply(lambda x: '360 мин' if x == '-' else x)
df_routes_inf.intervals = df_routes_inf.intervals.apply(lambda x:  x.replace('до ', '') if 'до 80 мин.' in x else x)

In [31]:
def parse_mints(mints):
    if '-' in mints:
        mints = [int(i.strip()) for i in mints.split('-')]
        return sum(mints)/len(mints)
    return int(mints.strip())

def parse_intervals(x):
    parts = x.split('.')
    try:
        if ':' in x:
            return {i.split(':')[0] : parse_mints(i.split(':')[1].replace(' мин', '')) for i in parts if ':' in i}
        return {'always': parse_mints(parts[0].replace(' мин', ''))}
    except:
        print(x)
df_routes_inf['intervals_dict'] = df_routes_inf.intervals.apply(parse_intervals)
# t = 'По рабочим дням: 10-20 мин.По выходным дням: 20-30 мин.В вечернее время (после 20 часов): 30 мин.'
# parse_intervals(t)

In [32]:
# set([j for i in df_routes_inf.schedule_days.values for j in i.values()])
# df_routes_inf.intervals.unique()
# df_routes_inf.head()

In [33]:
def schedule_intervals(x):
    keys = x.intervals_dict.keys()
    work_day_int, sunday_int, saturday_int, int_18, int_19, int_20 = [None for i in range(6)]  
    if 'always' in keys:
        work_day_int, sunday_int, saturday_int, int_18, int_19, int_20 = [x.intervals_dict['always'] for i in range(6)]
    if 'По рабочим дням' in keys:
        work_day_int = x.intervals_dict['По рабочим дням'] 
    if 'По выходным дням' in keys:
        sunday_int, saturday_int = x.intervals_dict['По выходным дням'], x.intervals_dict['По выходным дням'] 
    if 'По субботам' in keys:
        saturday_int = x.intervals_dict['По субботам']
    if 'По воскресеньям' in keys:
        sunday_int = x.intervals_dict['По воскресеньям']
    if 'В вечернее время (после 18 часов)' in keys:
        int_18, int_19, int_20 = [x.intervals_dict['В вечернее время (после 18 часов)'] for i in range(3)]
    if 'В вечернее время (после 19 часов)' in keys:
        int_19, int_20 = [x.intervals_dict['В вечернее время (после 19 часов)'] for i in range(2)]
    if 'В вечернее время (после 20 часов)' in keys:
        int_20 = x.intervals_dict['В вечернее время (после 20 часов)']
    return pd.Series([work_day_int, sunday_int, saturday_int, int_18, int_19, int_20])

df_routes_inf[['work_day_int', 'sunday_int', 'saturday_int', 'int_18', 'int_19', 'int_20']] = df_routes_inf.apply(schedule_intervals, axis=1)

In [34]:
df_routes_inf.head()

,number,type_title,schedule_days,intervals,vehicles,work_day,saturday,sunday,religion,arena,summer_weekend,summer_thu_tue,summer_wed_fri,intervals_dict,work_day_int,sunday_int,saturday_int,int_18,int_19,int_20
0,1,Автобусный,{'Ежедневно': ' с 6 до 22 часов'},По рабочим дням: 8-20 мин.По выходным дням: 16...,ЛиАЗ-5293.70,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None,"{'По рабочим дням': 14.0, 'По выходным дням': ...",14.0,25.5,25.5,NaN,40.0,40.0
1,3,Автобусный,{'Ежедневно': ' с 7 до 20 часов'},48-60 мин.,МАЗ-206,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]",None,None,None,None,None,{'always': 54.0},54.0,54.0,54.0,54.0,54.0,54.0
2,4,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},7-10 мин.,Автобусы малого класса,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None,{'always': 8.5},8.5,8.5,8.5,8.5,8.5,8.5
3,5д,Автобусный,{'Ежедневно': ' с 5 до 22 часов'},По рабочим дням: 10-20 мин.По выходным дням: 1...,ЛиАЗ-5293.70,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",None,None,None,None,None,"{'По рабочим дням': 15.0, 'По выходным дням': ...",15.0,19.5,19.5,25.5,25.5,25.5
4,6,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},По рабочим дням: 15-34 мин.По выходным дням: 1...,МАЗ-206,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None,"{'По рабочим дням': 24.5, 'По выходным дням': ...",24.5,48.0,48.0,NaN,NaN,NaN


Если указан диапазон или разные комплектации единици траспорта, то взят минимум.

Если указана номинальная и максимальная вместимость, то взят максимум.

Если указаны и 1, и 2 вагона, то взято среднее.

Где машины не указаны:
- от 1 до 3 машин: вместимость Scania
- от 3 до 6 машин: МАЗ-206, как самый распространённый
- от 6 и выше: маршрутка

Если несколько разных ТС на маршруте - взято среднее

In [35]:
pass_count = pd.read_excel('samara_transport_passengers.xlsx', header=None)
pass_count_dict = {i[0]: i[1] for i in pass_count.to_dict('records')}

In [36]:
def pass_counting(x):
    lst = []
    for k, v in pass_count_dict.items():
        if k in x:
            lst.append(v)
    return sum(lst)/len(lst)
df_routes_inf['pass_count'] = df_routes_inf.vehicles.apply(pass_counting)

In [37]:
df_routes_capacity = df_routes_inf.copy()
df_routes_capacity = df_routes_capacity.drop(columns=['schedule_days','intervals','vehicles', 'intervals_dict'])
df_routes_capacity.head()

,number,type_title,work_day,saturday,sunday,religion,arena,summer_weekend,summer_thu_tue,summer_wed_fri,work_day_int,sunday_int,saturday_int,int_18,int_19,int_20,pass_count
0,1,Автобусный,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None,14.0,25.5,25.5,NaN,40.0,40.0,116.0
1,3,Автобусный,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]",None,None,None,None,None,54.0,54.0,54.0,54.0,54.0,54.0,72.0
2,4,Автобусный,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None,8.5,8.5,8.5,8.5,8.5,8.5,22.0
3,5д,Автобусный,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",None,None,None,None,None,15.0,19.5,19.5,25.5,25.5,25.5,116.0
4,6,Автобусный,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None,24.5,48.0,48.0,NaN,NaN,NaN,72.0


In [38]:
def carrying(x, day, interval):
    lst = []
    capacity = x['pass_count']
    for i in range(5, 24, 1):
        try:
            if i in x[day]:
                if i == 18 and x['int_18'] > 0:
                    lst.append(capacity*60/x['int_18'])
                elif i == 19 and x['int_19'] > 0:
                    lst.append(capacity*60/x['int_19'])
                elif i == 20 and x['int_20'] > 0:
                    lst.append(capacity*60/x['int_20'])
                else:
                    lst.append(capacity*60/x[interval])
            else:
                lst.append(None)
        except TypeError:
            lst.append(None)
    return pd.Series(lst)

In [39]:
df_routes_capacity[['work_day_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'work_day', 'work_day_int'),axis=1)
df_routes_capacity[['saturday_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'saturday', 'saturday_int'),axis=1)
df_routes_capacity[['sunday_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'sunday', 'sunday_int'),axis=1)
df_routes_capacity[['religion_sun_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'religion', 'sunday_int'),axis=1)
df_routes_capacity[['religion_sat_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'religion', 'saturday_int'),axis=1)
df_routes_capacity[['religion_wd_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'religion', 'work_day_int'),axis=1)
df_routes_capacity[['arena_wd_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'arena', 'work_day_int'),axis=1)
df_routes_capacity[['arena_sun_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'arena', 'sunday_int'),axis=1)
df_routes_capacity[['arena_sat_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'arena', 'saturday_int'),axis=1)
df_routes_capacity[['summer_wed_fri_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'summer_wed_fri', 'work_day_int'),axis=1)
df_routes_capacity[['summer_thu_tue_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'summer_thu_tue', 'work_day_int'),axis=1)
df_routes_capacity[['summer_sun_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'summer_weekend', 'sunday_int'),axis=1)
df_routes_capacity[['summer_sat_{}'.format(i) for i in range(5, 24, 1)]] = df_routes_capacity.apply(
    lambda y: carrying(y, 'summer_weekend', 'saturday_int'),axis=1)

In [40]:
df_routes_capacity = df_routes_capacity.drop(columns=['work_day','saturday','sunday','religion','arena',
                                                      'summer_weekend','summer_thu_tue','summer_wed_fri',
                                                      'work_day_int','sunday_int','saturday_int',
                                                      'int_18','int_19','int_20','pass_count'])
df_routes_capacity.head()

,number,type_title,work_day_5,work_day_6,work_day_7,work_day_8,work_day_9,work_day_10,work_day_11,work_day_12,...,summer_sat_14,summer_sat_15,summer_sat_16,summer_sat_17,summer_sat_18,summer_sat_19,summer_sat_20,summer_sat_21,summer_sat_22,summer_sat_23
0,1,Автобусный,NaN,497.142857,497.142857,497.142857,497.142857,497.142857,497.142857,497.142857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Автобусный,NaN,NaN,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,Автобусный,NaN,155.294118,155.294118,155.294118,155.294118,155.294118,155.294118,155.294118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5д,Автобусный,464.0,464.000000,464.000000,464.000000,464.000000,464.000000,464.000000,464.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,Автобусный,NaN,176.326531,176.326531,176.326531,176.326531,176.326531,176.326531,176.326531,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df_routes_capacity.to_csv('routes_capacity.csv', index=False)

In [42]:
df_routes_capacity.type_title.unique()

array(['Автобусный', 'Трамвайный', 'Троллейбусный'], dtype=object)

In [43]:
def remap_title(x):
    if x == 'Автобусный':
        return 1
    if x == 'Трамвайный':
        return 3
    if x == 'Троллейбусный':
        return 4
df_routes_capacity['type'] = df_routes_capacity.type_title.apply(remap_title)
df_routes_capacity = df_routes_capacity.drop(columns='type_title')

In [44]:
df_geo_capacity = df_routes_perf.join(df_routes_capacity.set_index(['number', 'type']), on=['number','type_id'])
df_geo_capacity.head()

,number,type_id,geo,work_day_5,work_day_6,work_day_7,work_day_8,work_day_9,work_day_10,work_day_11,...,summer_sat_14,summer_sat_15,summer_sat_16,summer_sat_17,summer_sat_18,summer_sat_19,summer_sat_20,summer_sat_21,summer_sat_22,summer_sat_23
0,1,1,"[((53.384692, 50.169369), (53.384749, 50.16960...",NaN,497.142857,497.142857,497.142857,497.142857,497.142857,497.142857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,"[((53.187111, 50.121445), (53.18814, 50.122168...",NaN,497.142857,497.142857,497.142857,497.142857,497.142857,497.142857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21,1,"[((53.259224, 50.216663), (53.259209, 50.21667...",NaN,296.842105,296.842105,296.842105,296.842105,296.842105,296.842105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21,1,"[((53.213223, 50.291229), (53.213392, 50.29159...",NaN,296.842105,296.842105,296.842105,296.842105,296.842105,296.842105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27,1,"[((53.213391, 50.247921), (53.217799, 50.24232...",NaN,98.142857,98.142857,98.142857,98.142857,98.142857,98.142857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# def geo_dummies(x):
#     return pd.Series([1 if i in x.geo else 0 for i in global_edges])
# df_geo_capacity.apply(geo_dummies, axis=1)
col = list(df_geo_capacity.columns)
col.remove('number')
col.remove('type_id')
col.remove('geo')


def edge_capacity(x, edge):
    if edge in x.geo:
        return pd.Series(x[col].values)
    return pd.Series([0 for i in col])

itera = 1
edge_weights = {}
for i in global_edges:
    print(i, ' ',itera, '/', len(global_edges))
    edge_weights[i] = df_geo_capacity.apply(lambda y: edge_capacity(y, i), axis=1).sum().to_list()
    itera+= 1

((53.17861, 50.145116), (53.178574, 50.144932))   1 / 13310
((53.230659, 50.278336), (53.232807, 50.275488))   2 / 13310
((53.223391, 50.287776), (53.223816, 50.286864))   3 / 13310
((53.496866, 50.056607), (53.497173, 50.056795))   4 / 13310
((53.142111, 50.132257), (53.141629, 50.131581))   5 / 13310
((53.143922, 50.049926), (53.143703, 50.049673))   6 / 13310
((53.222906, 50.288585), (53.22301, 50.288436))   7 / 13310
((53.237857, 50.216627), (53.238367, 50.215984))   8 / 13310
((53.246244, 50.309452), (53.246615, 50.309495))   9 / 13310
((53.210476, 50.298915), (53.21054, 50.29874))   10 / 13310
((53.215937, 50.331405), (53.21608, 50.331646))   11 / 13310
((53.261915, 50.22414), (53.261754, 50.224224))   12 / 13310
((53.187184, 50.135397), (53.187082, 50.135756))   13 / 13310
((53.236367, 50.191908), (53.236178, 50.191553))   14 / 13310
((53.190607, 50.101302), (53.188209, 50.099679))   15 / 13310
((53.207382, 50.277687), (53.207564, 50.277903))   16 / 13310
((53.251502, 50.221933)

((53.333994, 50.301312), (53.335815, 50.301737))   134 / 13310
((53.301455, 50.225925), (53.298988, 50.226243))   135 / 13310
((53.443486, 50.24946), (53.443474, 50.249326))   136 / 13310
((53.111104, 50.141382), (53.110907, 50.141204))   137 / 13310
((53.219705, 50.165956), (53.219618, 50.165882))   138 / 13310
((53.444012, 50.251284), (53.444065, 50.251387))   139 / 13310
((53.116262, 50.228781), (53.116438, 50.229088))   140 / 13310
((53.165152, 50.076237), (53.165345, 50.074808))   141 / 13310
((53.110251, 50.068341), (53.107548, 50.071376))   142 / 13310
((53.107031, 50.134959), (53.107053, 50.135272))   143 / 13310
((53.223413, 50.234951), (53.223505, 50.234861))   144 / 13310
((53.190367, 50.113429), (53.190367, 50.11329))   145 / 13310
((53.205455, 50.279394), (53.206641, 50.278034))   146 / 13310
((53.181231, 50.104787), (53.181736, 50.102781))   147 / 13310
((53.147229, 50.139443), (53.147197, 50.139748))   148 / 13310
((53.142493, 50.144345), (53.143126, 50.14382))   149 / 1

((53.22547, 50.283204), (53.225371, 50.283417))   265 / 13310
((53.246674, 50.203376), (53.246574, 50.203392))   266 / 13310
((53.203319, 50.125959), (53.205177, 50.123608))   267 / 13310
((53.229171, 50.176802), (53.227958, 50.174975))   268 / 13310
((53.189909, 50.143295), (53.188933, 50.14389))   269 / 13310
((53.199473, 50.146338), (53.19727, 50.141076))   270 / 13310
((53.251196, 50.25768), (53.251111, 50.257705))   271 / 13310
((53.483082, 50.037933), (53.483201, 50.03761))   272 / 13310
((53.252102, 50.310347), (53.252918, 50.310439))   273 / 13310
((53.21656, 50.149053), (53.217718, 50.151728))   274 / 13310
((53.147197, 50.139748), (53.147128, 50.139972))   275 / 13310
((53.157187, 50.089084), (53.157073, 50.089769))   276 / 13310
((53.225371, 50.283417), (53.225272, 50.28363))   277 / 13310
((53.203662, 50.231659), (53.203795, 50.231972))   278 / 13310
((53.305894, 50.239267), (53.306521, 50.238508))   279 / 13310
((53.268628, 50.252726), (53.266666, 50.249149))   280 / 13310

((53.183958, 50.106841), (53.183675, 50.106643))   396 / 13310
((53.116264, 50.124008), (53.118477, 50.121524))   397 / 13310
((53.167602, 50.456979), (53.166886, 50.456953))   398 / 13310
((53.192704, 50.102824), (53.192746, 50.10295))   399 / 13310
((53.44394, 50.251164), (53.443848, 50.251))   400 / 13310
((53.407546, 50.360344), (53.407966, 50.357826))   401 / 13310
((53.083843, 50.329134), (53.083793, 50.33036))   402 / 13310
((53.192108, 50.211874), (53.191948, 50.212353))   403 / 13310
((53.169266, 50.075497), (53.168613, 50.075087))   404 / 13310
((53.209251, 50.132921), (53.209271, 50.133052))   405 / 13310
((53.408882, 50.142562), (53.409248, 50.142304))   406 / 13310
((53.193819, 50.150537), (53.193846, 50.150787))   407 / 13310
((53.450668, 50.125245), (53.451041, 50.125534))   408 / 13310
((53.20182, 50.228297), (53.197079, 50.221003))   409 / 13310
((53.235077, 50.314632), (53.235458, 50.315577))   410 / 13310
((53.22607, 50.281932), (53.225969, 50.282143))   411 / 13310


((53.323415, 50.306705), (53.323407, 50.306507))   528 / 13310
((53.385129, 50.163576), (53.38317, 50.165496))   529 / 13310
((53.203982, 50.159809), (53.203975, 50.159566))   530 / 13310
((53.215541, 50.291762), (53.215437, 50.291714))   531 / 13310
((53.247768, 50.248116), (53.249582, 50.247166))   532 / 13310
((53.1265, 50.171364), (53.12662, 50.171392))   533 / 13310
((53.177865, 50.192816), (53.177507, 50.193002))   534 / 13310
((53.206008, 50.122552), (53.206257, 50.122281))   535 / 13310
((53.195186, 50.123957), (53.194677, 50.123596))   536 / 13310
((53.18814, 50.122168), (53.186992, 50.121362))   537 / 13310
((53.149383, 50.393906), (53.151277, 50.395235))   538 / 13310
((53.155632, 50.456719), (53.155884, 50.456744))   539 / 13310
((53.510799, 50.14218), (53.510542, 50.14147))   540 / 13310
((53.228776, 50.250669), (53.228956, 50.250436))   541 / 13310
((53.364538, 50.200856), (53.364426, 50.200998))   542 / 13310
((53.32324, 50.304081), (53.323411, 50.303605))   543 / 13310


...

((53.175774, 50.077832), (53.175781, 50.078034))   12769 / 13310
((53.322682, 50.299185), (53.322698, 50.299284))   12770 / 13310
((53.424212, 50.317982), (53.422754, 50.321729))   12771 / 13310
((53.200108, 50.132092), (53.199865, 50.131595))   12772 / 13310
((53.22127, 50.266877), (53.224161, 50.273167))   12773 / 13310
((53.435256, 50.124534), (53.435663, 50.124664))   12774 / 13310
((53.349523, 50.216042), (53.349594, 50.216033))   12775 / 13310
((53.253258, 50.364379), (53.253236, 50.364246))   12776 / 13310
((53.212372, 50.189177), (53.21138, 50.189244))   12777 / 13310
((53.345777, 50.243256), (53.345669, 50.24339))   12778 / 13310
((53.189012, 50.138372), (53.189116, 50.137932))   12779 / 13310
((53.206612, 50.235136), (53.208431, 50.239182))   12780 / 13310
((53.258863, 50.22397), (53.258922, 50.224209))   12781 / 13310
((53.257134, 50.221025), (53.258751, 50.223686))   12782 / 13310
((53.232645, 50.1982), (53.232707, 50.197914))   12783 / 13310
((53.202346, 50.282883), (53.20

((53.381258, 50.315706), (53.381426, 50.31576))   12896 / 13310
((53.191204, 50.170286), (53.191216, 50.170228))   12897 / 13310
((53.21979, 50.157443), (53.21979, 50.157675))   12898 / 13310
((53.257342, 50.244445), (53.258354, 50.243327))   12899 / 13310
((53.19206, 50.205653), (53.192085, 50.206541))   12900 / 13310
((53.238792, 50.284437), (53.239665, 50.280455))   12901 / 13310
((53.114923, 50.22542), (53.114869, 50.225151))   12902 / 13310
((53.219485, 50.155833), (53.219025, 50.154729))   12903 / 13310
((53.523411, 50.071915), (53.522779, 50.071354))   12904 / 13310
((53.242717, 50.209509), (53.242892, 50.20974))   12905 / 13310
((53.090497, 50.349424), (53.090143, 50.349396))   12906 / 13310
((53.121529, 50.213321), (53.122175, 50.21199))   12907 / 13310
((53.10321, 50.077292), (53.102781, 50.077004))   12908 / 13310
((53.185545, 50.146802), (53.185616, 50.146572))   12909 / 13310
((53.477254, 50.11659), (53.477638, 50.116469))   12910 / 13310
((53.197535, 50.140306), (53.19745

((53.183776, 50.076329), (53.183037, 50.075871))   13023 / 13310
((53.249339, 50.247198), (53.248516, 50.247603))   13024 / 13310
((53.251323, 50.36012), (53.251305, 50.359853))   13025 / 13310
((53.228714, 50.305043), (53.229507, 50.304145))   13026 / 13310
((53.424283, 50.35632), (53.4239, 50.355878))   13027 / 13310
((53.358519, 50.206815), (53.357033, 50.208275))   13028 / 13310
((53.18267, 50.110962), (53.182641, 50.111558))   13029 / 13310
((53.227888, 50.194678), (53.228291, 50.194105))   13030 / 13310
((53.184897, 50.089615), (53.182756, 50.088231))   13031 / 13310
((53.234639, 50.260204), (53.234609, 50.260339))   13032 / 13310
((53.196392, 50.129021), (53.196529, 50.128857))   13033 / 13310
((53.216107, 50.25612), (53.216174, 50.256291))   13034 / 13310
((53.230106, 50.249916), (53.226473, 50.24201))   13035 / 13310
((53.187348, 50.299897), (53.186372, 50.300984))   13036 / 13310
((53.280981, 50.228911), (53.28092, 50.22892))   13037 / 13310
((53.14733, 50.027663), (53.146499

((53.276582, 50.227821), (53.276497, 50.227764))   13151 / 13310
((53.172373, 50.191611), (53.173037, 50.194194))   13152 / 13310
((53.092306, 50.150082), (53.092176, 50.149939))   13153 / 13310
((53.370089, 50.189223), (53.370049, 50.189354))   13154 / 13310
((53.189745, 50.131993), (53.189767, 50.131868))   13155 / 13310
((53.267106, 50.249405), (53.26904, 50.252976))   13156 / 13310
((53.101257, 50.405002), (53.102632, 50.405992))   13157 / 13310
((53.324146, 50.304837), (53.323531, 50.303836))   13158 / 13310
((53.211448, 50.124533), (53.21266, 50.127069))   13159 / 13310
((53.276764, 50.227935), (53.276667, 50.227875))   13160 / 13310
((53.255944, 50.219018), (53.258616, 50.217067))   13161 / 13310
((53.192935, 50.122395), (53.192792, 50.122231))   13162 / 13310
((53.190168, 50.130292), (53.189795, 50.131618))   13163 / 13310
((53.257707, 50.204385), (53.257445, 50.204106))   13164 / 13310
((53.430608, 50.118415), (53.429544, 50.119071))   13165 / 13310
((53.184748, 50.108161), (5

((53.11111, 50.082303), (53.10718, 50.079818))   13278 / 13310
((53.236618, 50.201906), (53.23639, 50.201623))   13279 / 13310
((53.284771, 50.282205), (53.284364, 50.281921))   13280 / 13310
((53.299166, 50.243226), (53.302539, 50.241455))   13281 / 13310
((53.251292, 50.358461), (53.251295, 50.358418))   13282 / 13310
((53.402856, 50.147197), (53.401581, 50.148118))   13283 / 13310
((53.074416, 50.356928), (53.072609, 50.359558))   13284 / 13310
((53.147451, 50.135206), (53.147472, 50.135057))   13285 / 13310
((53.20666, 50.302231), (53.206735, 50.302173))   13286 / 13310
((53.34657, 50.226814), (53.346569, 50.22672))   13287 / 13310
((53.321917, 50.296272), (53.321934, 50.296209))   13288 / 13310
((53.166906, 50.073898), (53.167091, 50.074046))   13289 / 13310
((53.218005, 50.26457), (53.218349, 50.264118))   13290 / 13310
((53.24629, 50.203055), (53.246776, 50.203479))   13291 / 13310
((53.303056, 50.289856), (53.303056, 50.289856))   13292 / 13310
((53.274347, 50.19057), (53.27442

In [46]:
edge_weights_df = pd.DataFrame.from_dict(edge_weights)
# edge_weights_df = pd.read_csv('edge_weights_df.csv', index_col=0, header=None, low_memory=False).reset_index().drop(columns=0)
edge_weights_df.head()

,"(53.17861, 50.145116)","(53.230659, 50.278336)","(53.223391, 50.287776)","(53.496866, 50.056607)","(53.142111, 50.132257)","(53.143922, 50.049926)","(53.222906, 50.288585)","(53.237857, 50.216627)","(53.246244, 50.309452)","(53.210476, 50.298915)",...,"(53.213354, 50.142225)","(53.219476, 50.262903)","(53.197493, 50.221397)","(53.203002, 50.141237)","(53.279572, 50.228974)","(53.185539, 50.2306)","(53.466788, 50.126972)","(53.246484, 50.236666)","(53.104518, 50.162654)","(53.216867, 50.221214)"
,"(53.178574, 50.144932)","(53.232807, 50.275488)","(53.223816, 50.286864)","(53.497173, 50.056795)","(53.141629, 50.131581)","(53.143703, 50.049673)","(53.22301, 50.288436)","(53.238367, 50.215984)","(53.246615, 50.309495)","(53.21054, 50.29874)",...,"(53.213857, 50.143322)","(53.219401, 50.262846)","(53.196812, 50.220343)","(53.202991, 50.140595)","(53.278671, 50.228817)","(53.184992, 50.229857)","(53.468213, 50.125531)","(53.246556, 50.23685)","(53.103448, 50.162266)","(53.216907, 50.221401)"
0,0.0,0.0000,1342.222222,0.0,254.684526,758.196721,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,2820.000000,0.000000,0.000000,0.000000,0.0,66.000000,0.000
1,0.0,56.5625,2862.222222,0.0,254.684526,758.196721,1626.666667,1279.5,349.571429,47.5,...,651.467952,2117.142857,1003.333333,4297.942857,714.862135,148.965517,149.810853,0.0,236.500000,178.125
2,80.0,56.5625,2862.222222,0.0,254.684526,758.196721,1626.666667,1279.5,349.571429,47.5,...,651.467952,2117.142857,1003.333333,4297.942857,714.862135,148.965517,149.810853,0.0,243.833333,178.125
3,80.0,0.0000,2862.222222,0.0,254.684526,758.196721,1626.666667,1279.5,349.571429,47.5,...,651.467952,2117.142857,1003.333333,4297.942857,714.862135,148.965517,149.810853,0.0,243.833333,178.125
4,80.0,0.0000,1912.222222,0.0,254.684526,758.196721,1626.666667,1279.5,349.571429,47.5,...,651.467952,2117.142857,1003.333333,4297.942857,714.862135,148.965517,149.810853,0.0,243.833333,0.000


In [47]:
len(edge_weights_df.columns)

13310

In [48]:
edge_weights_df.to_csv('edge_weights_df.csv', sep=';')

In [ ]:
edge_w_df = edge_weights_df.transpose()
edge_w_df.columns = ['edge 1', 'edge 2'] + col
edge_w_df = edge_w_df.reset_index()
edge_w_df = edge_w_df.drop(columns = 'index')
edge_w_df.head()

In [ ]:
edge_w_df[col] = edge_w_df[col].astype(float)
len(edge_w_df)

In [ ]:
edge_w_df.work_day_10.describe()

In [ ]:
%matplotlib inline
edge_w_df.work_day_10[edge_w_df.work_day_10 > 0].hist()

In [ ]:
def choose_color(x):
    if x > 8000:
        return '#331900'
    if x > 6000:
        return '#663300'
    if x > 4000:
        return '#994c00'
    if x > 2000:
        return '#cc6600'
    if x > 1000:
        return '#ff8000'
    if x > 500:
        return '#ff9933'
    if x > 250:
        return '#ffb266'
    if x > 100:
        return '#ffcc99'
    return '#ffffff'
    

In [ ]:
work_day_10_df = edge_w_df[edge_w_df.work_day_10 > 0][['edge 1', 'edge 2','work_day_10']].values
work_day_10_dict = {}
for i in work_day_10_df:
    k1 = i[0].replace('(', '').replace(')', '').split(', ')
    k2 = i[1].replace('(', '').replace(')', '').split(', ')
    k = ((float(k1[0]), float(k1[1])),(float(k2[0]), float(k2[1])))
    v = choose_color(i[2])
    work_day_10_dict[k] = v

In [ ]:
points = {j for i in list(global_edges) for j in i}

ave_lat = sum(p[0] for p in points)/len(points)
ave_lon = sum(p[1] for p in points)/len(points)

# Load map centred on average coordinates
my_map = folium.Map(location=[ave_lat, ave_lon], zoom_start=10)

#fadd lines
for k,v in work_day_10_dict.items():
    folium.PolyLine(k, color=v, weight=3, opacity=1).add_to(my_map)

# Save map
my_map.save("./work_day_10.html")
# my_map